<a href="https://colab.research.google.com/github/stazam/DTSE-project/blob/main/DTSE_steam_data_reviews_model_building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Model bulding**

First we will start by preprocessing text variables in datasets:

1. checking english language - we will be modelling only english written variables
2. removing some special characters
3. 

We will try to use various models for user_review category prediction. Types of models which we will consider:

1. Bidirectional LSTM layer (+ CNN layer) with embedding layer without removing stop words and lematization 
2. Bidirectional LSTM layer (+ CNN layer) with embedding layer with using a stop wrods and lematization 
3. BERT - transformer for text classification without pretrained embeddings without removing stop words and lematization. 
4. BERT - transformer for text classification with pretrained embeddings without removing stop words and lematization.

Also we will use stacking method to incorporate information from varibales **year**. 

At the end we create **predict_pipeline** function for prediction, which will consist of preprocessing and prediction part.



In [3]:
import tensorflow as tf
import keras
import sys
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import metrics

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

from keras import applications
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional
from keras.layers import Convolution2D, MaxPooling2D,BatchNormalization,GlobalAveragePooling1D, Flatten, Dropout
from keras import optimizers

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

!pip install langdetect
from langdetect import detect

from google.colab import drive
drive.mount('/content/drive')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
     |████████████████████████████████| 981 kB 4.6 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=a07ed3aec9f79e4769477dc58788fdcab8b6b6161f13c295d1bef1824ddc97fd
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
sys.path.append('/content/drive/MyDrive/DTSE-data/Python-files')
from help_functions import *

In [39]:
df_train_merged = pd.read_pickle("/content/drive/MyDrive/DTSE-data/data-files/df_train_merged.pkl")
df_test_merged = pd.read_pickle("/content/drive/MyDrive/DTSE-data/data-files/df_test_merged.pkl")

# 1. **Bidirectional LSTM** (+ CNN) without removing stop words and lematization




# **Text** preprocessing

We will check for language of string variables. Since we have seen during **game_overview** file inspection that both *developer, publisher* and also *title* are english words, we do not need to check them. But we have to inspect **user_review** for english language.

In [6]:
def check_language(x):
    
    try:
      return detect(x) 
    except:
        return None

df_train_merged['languge'] = df_train_merged.user_review.apply(lambda x: check_language(x))

In [7]:
print('{} rows from train dataset in different language were removed'.format(sum(df_train_merged.languge.values != 'en')))
df_train_merged = df_train_merged.loc[df_train_merged.languge == 'en',:]

215 rows from train dataset in different language were removed


We can see we loose 215 rows with user_reviews in differnt languages.

In [1]:
import re 

In [ ]:
#symbols = set([y for x in df_train_merged.user_review.values for y in x if not y.isalnum()])
#print(symbols)
symbols_up = {'⭐', '`', 'ี', '／', '“', '⟶', '‘', '˘', '\u2007', '□', '♬', '\xad', '«', '้', '☢', '[', '¯', '＼', ']', '☠', 'Ⓚ', 'ั', '¶', '€', '⌐', '^', '‹', '⁀', '･', '£', '%', '=', ' ', '*', '¬', '▀', '━', '♪', '⌬', '∞', '&', ';', '”', '・', '▒', '+', '☐', '{', '¨', '⌠', '☹', '\xa0', '｜', '~', '？', '」', '¿', '¢', '่', '|', '◕', '@', '‡', '⟵', '☻', '·', '≠', '´', '☑', '✎', '۞', '͠', '˚', '✖', '●', '✦', '✰', '⋎', '͜', '✫', '■', '✩', '،', '┐', '►', "'", '_', '→', '❤', '§', '︵', '░', '✌', '▽', '•', '！', '▲', '█', '?', '▐', '˜', '͟', '}', '♥', '▄', '☣', '＿', '〃', '\u3000', '＾', '☼', '★', '®', '┌', '¤', '☺', '\\', '▌', 'ู', '「', '¡', '’', '▼', '¦', '°', '┳', '…', '#', '±', '～', '\u200b', '>', '⚠', '⌒', '‰', '<', '©', '✓', '‵', '™',  'ื', '\ufeff', 'ⓘ', '。', '‿', '︻', '͡', 'Ⓞ', '↑', '/', '¥', '$', '⊂', '═', '♫', '☆', 'ิ', '†', '̶', '↓', '✔', '┻'}

In [ ]:
' '.join(re.findall('[A-Z][^A-Z]*', 'TakToto je super SkuskaOdvahy'))

In [ ]:
s.replace("'s"," is").replace("'re"," are").replace("'ll",' will')

In [ ]:
for i in range(20,100):
  for x in symbols_up:
    df_train_merged.user_review[i] = df_train_merged.user_review[i].replace(x,' ')
  print(df_train_merged.user_review[i]) 

In [ ]:
text = re.sub(r"\s{2,}", " ",a)

In [11]:
df_train_merged['text'] = df_train_merged.title + df_train_merged.developer + df_train_merged.publisher + df_train_merged.tags.apply(lambda x: x[1:-1]) + df_train_merged.overview + df_train_merged.user_review
df_test_merged['text'] = df_test_merged.title + df_test_merged.developer + df_test_merged.publisher + df_test_merged.tags.apply(lambda x: x[1:-1]) + df_test_merged.overview + df_test_merged.user_review

X = df_train_merged
y = X.user_suggestion.values


We will start by adding a variable **text** which will be composed of variables *title, developer, publisher, tags, overview, user_review.*

I also consider training two separate models, first just on *user_reveiew* and second on the rest of mentioned variables and then stack results rogether, but I belive it wouldn't improve performance of model, because tags, developer, publisher, title are the same rows in the data set, once when the targer variable **user_suggestion** is 1 and once when 0, so the model would be confused. 

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [32]:
preprocessed_input = []
for x in X_NN:
  X_NN = X_NN.replace('\'','')
  X_NN

AttributeError: ignored

In [ ]:
X_train_NN = X_train.text.values
X_test_NN = X_test.text.values

In [ ]:
print("The longest sequence is {} words long.".format(max([len(x) for x in X_train_NN])))
pd.DataFrame([len(x) for x in X_train_NN]).describe()

In [ ]:
max_length = 5000
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
vocab_size = 10000
#training_size = 20000

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train_NN)

print('Vocabulary is {} words large'.format(len(tokenizer.word_index)))

In [ ]:
sorted(tokenizer.word_counts.items(), key = lambda t: t[1])[47000:]

In [ ]:
X_train_NN = tokenizer.texts_to_sequences(X_train_NN)
X_train_padded_NN = np.array(pad_sequences(X_train_NN, maxlen=max_length, padding=padding_type, truncating=trunc_type))

X_test_NN = tokenizer.texts_to_sequences(X_test_NN)
X_test_padded_NN = np.array(pad_sequences(X_test_NN, maxlen=max_length, padding=padding_type, truncating=trunc_type))


In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model = tf.keras.Sequential()

model.add(Embedding(vocab_size,240, input_length = max_length))
model.add(Bidirectional(keras.layers.LSTM(64, return_sequences=True)))
model.add(Bidirectional(keras.layers.LSTM(32, return_sequences=True)))
model.add(Flatten()),
Dropout(0.5),
model.add(Dense(1,activation = 'sigmoid'))


model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics = METRICS)

model.summary()

In [ ]:
%%time 

history = model.fit(X_train_padded_NN, y_train, epochs=1, validation_data=(X_test_padded_NN, y_test), verbose = 1)

In [ ]:
X_train_pred = model.predict(X_train_padded_NN)
X_test_pred = model.predict(X_test_padded_NN)

In [ ]:
from sklearn.impute import KNNImputer
import numpy as np

def knn_inputer(X):

    X[X.isnull()] = np.nan

    imputer = KNNImputer(n_neighbors=2)
    X_im = imputer.fit_transform(X)
    return pd.DataFrame(X_im, columns = list(X.columns))

def process_dat(X):
  
  X_1 = X.loc[:,['title','developer','publisher']]
  X_1 = pd.get_dummies(X_1)
  X_1 = pd.concat([X_1,X.loc[:,['review_id','year']]], axis = 1)

  return X_1

X_trainm = process_dat(X_train)
X_trainm = knn_inputer(X_trainm)
X_trainm['predictions'] = [x[0] for x in X_train_pred.tolist()]

X_testm = process_dat(X_test)
X_testm = knn_inputer(X_testm)
X_testm['predictions'] = [x[0] for x in X_test_pred.tolist()]

print(X_trainm.isnull().sum())
print(X_testm.isnull().sum())

# 2. **Bidirectional LSTM** (+ CNN) with removing stop words and lematization


In [22]:
X_train.text.values[100].replace('\'','')

'Dota 2Valve Valve \'Free to Play\', \'MOBA\', \'Strategy\', \'Multiplayer\', \'Team-Based\', \'Action\', \'e-sports\', \'Online Co-Op\', \'Competitive\', \'PvP\', \'RTS\', \'Difficult\', \'RPG\', \'Fantasy\', \'Tower Defense\', \'Co-op\', \'Character Customization\', \'Replay Value\', \'Action RPG\', \'Simulation\'The most-played game on Steam.Every day, millions of players worldwide enter battle as one of over a hundred Dota heroes. And no matter if it\'s their 10th hour of play or 1,000th, there\'s always something new to discover. With regular updates that ensure a constant evolution of gameplay, features, and heroes, Dota 2 has truly taken on a life of its own.One Battlefield. Infinite Possibilities.When it comes to diversity of heroes, abilities, and powerful items, Dota boasts an endless array—no two games are the same. Any hero can fill multiple roles, and there\'s an abundance of items to help meet the needs of each game. Dota doesn\'t provide limitations on how to play, it em

In [ ]:
def process_test(x):
  
  lemmatizer = WordNetLemmatizer()
  
  x = x.replace("'re",'are')
  x = x.replace("'s",'is')
  x = x.replace("'n",'not')
  x = x.replace("'n",'not')

  stop_words = set(stopwords.words('english'))
  word_tokens = word_tokenize(x)

  filtered_sentence = [lemmatizer.lemmatize(w) for w in word_tokens if not w.lower() in stop_words]
  
  return " ".join(filtered_sentence)

In [ ]:
for x in skuska:
  print(x)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
df_train_merged['text'] = df_train_merged.tags.apply(lambda x: x[1:-1]) + df_train_merged.overview + df_train_merged.user_review

X = df_train_merged
y = X.user_suggestion.values
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train_NN = X_train.text.values
X_test_NN = X_test.text.values

In [ ]:
print("The longest sequence is {} words long.".format(max([len(x) for x in X_train_NN])))
pd.DataFrame([len(x) for x in X_train_NN]).describe()

In [ ]:
max_length = 5000
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
vocab_size = 10000
#training_size = 20000

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train_NN)

print('Vocabulary is {} words large'.format(len(tokenizer.word_index)))

In [ ]:
sorted(tokenizer.word_counts.items(), key = lambda t: t[1])[47000:]

In [ ]:
X_train_NN = tokenizer.texts_to_sequences(X_train_NN)
X_train_padded_NN = np.array(pad_sequences(X_train_NN, maxlen=max_length, padding=padding_type, truncating=trunc_type))

X_test_NN = tokenizer.texts_to_sequences(X_test_NN)
X_test_padded_NN = np.array(pad_sequences(X_test_NN, maxlen=max_length, padding=padding_type, truncating=trunc_type))

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model = tf.keras.Sequential()

model.add(Embedding(vocab_size,240, input_length = max_length))
model.add(Bidirectional(keras.layers.LSTM(64, return_sequences=True)))
model.add(Bidirectional(keras.layers.LSTM(32, return_sequences=True)))
model.add(Flatten()),
Dropout(0.5),
model.add(Dense(1,activation = 'sigmoid'))


model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics = METRICS)

model.summary()

In [ ]:
history = model.fit(X_train_padded_NN, y_train, epochs=1, validation_data=(X_test_padded_NN, y_test), verbose = 1)

In [ ]:
X_train_pred = model.predict(X_train_padded_NN)
X_test_pred = model.predict(X_test_padded_NN)

In [ ]:
from sklearn.metrics import confusion_matrix

#first is category 0 then catgory 1. Check:
#np.logical_and((y_test ==1),np.array([x[0] for x in (round(X_test_p) == 1).values])).sum()

results = confusion_matrix(y_test, round(X_test_p))
print(results)

import seaborn as sns

ax = sns.heatmap(results, annot=True, cmap='Blues')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
from sklearn.impute import KNNImputer
import numpy as np

def knn_inputer(X):

    X[X.isnull()] = np.nan

    imputer = KNNImputer(n_neighbors=2)
    X_im = imputer.fit_transform(X)
    return pd.DataFrame(X_im, columns = list(X.columns))

def process_dat(X):
  
  X_1 = X.loc[:,['title','developer','publisher']]
  X_1 = pd.get_dummies(X_1)
  X_1 = pd.concat([X_1,X.loc[:,['review_id','year']]], axis = 1)

  return X_1

X_trainm = process_dat(X_train)
X_trainm = knn_inputer(X_trainm)
X_trainm['predictions'] = [x[0] for x in X_train_pred.tolist()]

X_testm = process_dat(X_test)
X_testm = knn_inputer(X_testm)
X_testm['predictions'] = [x[0] for x in X_test_pred.tolist()]

print(X_trainm.isnull().sum())
print(X_testm.isnull().sum())